# Problema de clasificación de imágenes

## Preparación del entorno

In [ ]:
!pip install wandb

In [ ]:
!pip install torchcam

### Importaciones

In [ ]:
import os
import random
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
from collections import defaultdict
import wandb

In [ ]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
import random

### Configuración del dispositivo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Preparación del dataset

### Definición de transformaciones

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

### Carga del dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/SIGE-P2/starting-package/data x20"

In [ ]:
dataset = datasets.ImageFolder(root=data_path, transform=transform)

### División: Entrenamiento y validación

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Análisis Exploratorio

In [ ]:
# Número de clases
print(f"Número de clases: {len(dataset.classes)}")

In [ ]:
# Número de imágenes
print(f"Número total de imágenes: {len(dataset)}")

In [ ]:
# Conteo por clase
class_counts = Counter([label for _, label in dataset.samples])
for i, count in class_counts.items():
    print(f"{dataset.classes[i]}: {count} imágenes")

In [ ]:
# Limpiar nombres de clases: quitar prefijo numérico y reemplazar guiones bajos
def clean_name(class_name):
    name_without_prefix = class_name.split('.', 1)[-1]
    return name_without_prefix.replace('_', ' ')

clean_names = {i: clean_name(name) for i, name in enumerate(dataset.classes)}
label_clean_names = [clean_names[label] for _, label in dataset.samples]
label_count = Counter(label_clean_names)

In [ ]:
# Distribución por clases
plt.figure(figsize=(12, 6))
plt.bar(label_count.keys(), label_count.values(), color='skyblue')
plt.xticks(rotation=90)
plt.title("Distribución de clases")
plt.xlabel("Clase")
plt.ylabel("Número de imágenes")
plt.tight_layout()
plt.show()


In [ ]:
# Crear diccionario con nombres limpios
clean_names = {i: clean_name(name) for i, name in enumerate(dataset.classes)}

# Agrupar rutas de imágenes por clase
class_images = defaultdict(list)
for path, label in dataset.samples:
    class_images[label].append(path)

# Mostrar una imagen aleatoria por clase
num_classes = len(dataset.classes)
cols = 5
rows = (num_classes + cols - 1) // cols
plt.figure(figsize=(15, rows * 3))

for label in range(num_classes):
    img_path = random.choice(class_images[label])
    img = Image.open(img_path).convert("RGB")
    plt.subplot(rows, cols, label + 1)
    plt.imshow(img)
    plt.title(clean_names[label], fontsize=9)
    plt.axis("off")

plt.tight_layout()
plt.show()


## Entrenamiento

### Funciones

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, scheduler, num_epochs=10):
    train_loss_history, val_loss_history = [], []
    train_acc_history, val_acc_history = [], []

    best_val_acc = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch + 1}/{num_epochs}")
        print("-" * 30)

        # === ENTRENAMIENTO ===
        model.train()
        running_loss, running_corrects = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = outputs.argmax(1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

        # === VALIDACIÓN ===
        model.eval()
        val_loss_total, val_corrects = 0.0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = outputs.argmax(1)

                val_loss_total += loss.item() * inputs.size(0)
                val_corrects += (preds == labels).sum().item()

        val_loss = val_loss_total / len(val_loader.dataset)
        val_acc = val_corrects / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc)

        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        scheduler.step(val_loss)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    print(f"\n✅ Mejor Accuracy de Validación: {best_val_acc:.4f}")
    model.load_state_dict(best_model_state)
    return model, train_loss_history, val_loss_history, train_acc_history, val_acc_history


In [ ]:
def plot_training(train_loss, val_loss, train_acc, val_acc):
    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 5))

    # Curva de pérdida
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label='Entrenamiento')
    plt.plot(epochs, val_loss, label='Validación')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Pérdida durante el entrenamiento')
    plt.legend()

    # Curva de accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label='Entrenamiento')
    plt.plot(epochs, val_acc, label='Validación')
    plt.xlabel('Época')
    plt.ylabel('Accuracy')
    plt.title('Accuracy durante el entrenamiento')
    plt.legend()

    plt.tight_layout()
    plt.show()


### Modelos

#### resnet18

In [ ]:
# Cargar modelo preentrenado
model18 = models.resnet18(pretrained=True)

# Congelar capas convolucionales
for param in model18.parameters():
    param.requires_grad = False

# Reemplazar la capa final para adaptarla a nuestras clases
num_ftrs = model18.fc.in_features
model18.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Mover el modelo al dispositivo adecuado (GPU/CPU)
model18 = model18.to(device)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model18.fc.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Entrenamiento del modelo
model, train_loss, val_loss, train_acc, val_acc = train_model(
    model18, criterion, optimizer, train_loader, val_loader, scheduler, num_epochs=10
)

# Visualización de resultados
plot_training(train_loss, val_loss, train_acc, val_acc)


#### resnet50

In [ ]:
# Cargar modelo preentrenado
model50 = models.resnet50(pretrained=True)

# Congelar capas convolucionales
for param in model50.parameters():
    param.requires_grad = False

# Reemplazar la capa final para adaptarla a nuestras clases
num_ftrs = model50.fc.in_features
model50.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Mover el modelo al dispositivo adecuado (GPU/CPU)
model50 = model50.to(device)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model50.fc.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Entrenamiento del modelo
model, train_loss, val_loss, train_acc, val_acc = train_model(
    model50, criterion, optimizer, train_loader, val_loader, scheduler, num_epochs=10
)

# Visualización de resultados
plot_training(train_loss, val_loss, train_acc, val_acc)


## Aumento a 200 categorías

### Carga del dataset

In [ ]:
data_path = "/content/drive/MyDrive/SIGE-P2/starting-package/data x200"
dataset = datasets.ImageFolder(root=data_path, transform=transform)

### División: Entrenamiento y validación

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Modelo

In [ ]:
# Cargar modelo preentrenado
model50 = models.resnet50(pretrained=True)

# Congelar capas convolucionales
for param in model50.parameters():
    param.requires_grad = False

# Reemplazar la capa final para adaptarla a nuestras clases
num_ftrs = model50.fc.in_features
model50.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Mover el modelo al dispositivo adecuado (GPU/CPU)
model50 = model50.to(device)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model50.fc.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Entrenamiento del modelo
model, train_loss, val_loss, train_acc, val_acc = train_model(
    model50, criterion, optimizer, train_loader, val_loader, scheduler, num_epochs=10
)

# Visualización de resultados
plot_training(train_loss, val_loss, train_acc, val_acc)

## Mejoras

### Weights And Biases

In [ ]:
wandb.login()

In [ ]:
wandb.init(
    project="clasificador-cub200",
    name="resnet50-x200",
    config={
        "modelo": "resnet50",
        "fine_tuning": "layer4",
        "clases": 20,
        "batch_size": 32,
        "epochs": 10,
        "optimizer": "AdamW",
        "lr": 1e-4,
        "early_stopping": True,
        "label_smoothing": 0.1,
        "data_augmentation": True
    }
)

### Data augmentation

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

### Early Stopping

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, scheduler=None, num_epochs=10):
    train_loss_history, val_loss_history = [], []
    train_acc_history, val_acc_history = [], []

    best_val_acc = 0.0
    best_model_state = None

    # Early Stopping
    patience = 3
    epochs_no_improve = 0

    wandb.watch(model)

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch + 1}/{num_epochs}")
        print("-" * 30)

        # === ENTRENAMIENTO ===
        model.train()
        running_loss, running_corrects = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = outputs.argmax(1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

        # === VALIDACIÓN ===
        model.eval()
        val_loss_total, val_corrects = 0.0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = outputs.argmax(1)

                val_loss_total += loss.item() * inputs.size(0)
                val_corrects += (preds == labels).sum().item()

        val_loss = val_loss_total / len(val_loader.dataset)
        val_acc = val_corrects / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc)

        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        # Scheduler opcional (ReduceLROnPlateau, etc.)
        if scheduler:
            scheduler.step(val_loss)

        # === EARLY STOPPING ===
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"🕓 Sin mejora en val_acc durante {epochs_no_improve} épocas")
            if epochs_no_improve >= patience:
                print("⏹️ Early stopping activado")
                break

    print(f"\n✅ Mejor Accuracy de Validación: {best_val_acc:.4f}")
    model.load_state_dict(best_model_state)
    return model, train_loss_history, val_loss_history, train_acc_history, val_acc_history

### Label Smoothing

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

### Entrenamiento

In [ ]:
# Cargar modelo preentrenado
model50 = models.resnet50(pretrained=True)

# Congelar capas convolucionales
for param in model50.parameters():
    param.requires_grad = False

# Reemplazar la capa final para adaptarla a nuestras clases
num_ftrs = model50.fc.in_features
model50.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Mover el modelo al dispositivo adecuado (GPU/CPU)
model50 = model50.to(device)

optimizer = torch.optim.Adam(model50.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

In [ ]:
# Entrenamiento del modelo
model, train_loss, val_loss, train_acc, val_acc = train_model(
    model50, criterion, optimizer, train_loader, val_loader, scheduler, num_epochs=10
)

In [ ]:
# Visualización de resultados
plot_training(train_loss, val_loss, train_acc, val_acc)

In [ ]:
wandb.finish()

## Explicabilidad

In [ ]:
# Seleccionamos 5 imagenes aleatorias de una clase
class_name = "010.Red_winged_Blackbird"
class_path = os.path.join(data_path, class_name)
class_images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
selected_images = random.sample(class_images, 5)

In [ ]:
import torch.nn.functional as F
from torchvision.transforms.functional import to_pil_image
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask

# Transformación fija (sin randomizaciones)
fixed_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

cam_extractor = GradCAM(model, target_layer="layer3")

plt.figure(figsize=(15, 8))
for idx, path_img in enumerate(selected_images):
    image = Image.open(path_img).convert("RGB")
    input_tensor = fixed_transform(image).unsqueeze(0).to(device)
    input_tensor.requires_grad_()

    with torch.set_grad_enabled(True):
        output = model(input_tensor)
        class_idx = output.argmax().item()

        # cam: ya viene en forma [1, 14, 14]
        cam = cam_extractor(class_idx, output)[0]  # [1, 14, 14]

        # Redimensionar (ya tiene forma [1, H, W] → añadimos batch dim)
        cam_resized = F.interpolate(cam.unsqueeze(0), size=(224, 224), mode="bilinear", align_corners=False)[0]

        # Normalizar al rango [0,1]
        cam_resized = cam_resized - cam_resized.min()
        cam_resized = cam_resized / (cam_resized.max() + 1e-8)

        # Convertir a PIL
        cam_img = to_pil_image(cam_resized.cpu())

        # Superponer heatmap sobre la imagen original redimensionada
        result = overlay_mask(image.resize((224, 224)), cam_img, alpha=0.5)

    plt.subplot(1, 5, idx + 1)
    plt.imshow(result)
    plt.title(f"Pred: {dataset.classes[class_idx]}", fontsize=9)
    plt.axis("off")

plt.suptitle(f"Grad-CAM sobre imágenes aleatorias de la clase: {class_name}", fontsize=14)
plt.tight_layout()
plt.show()
